# Gavin Kendal-Freedman

## Research question/interests

**My main interest is how different chemicals effect air quality ratings, specifically the main hazardous components in fuels for motor vehicles or stoves like natural gas/gasoline/diesel fuels (benzene, toluene, propane, methane, sulfurs/oxides of nitrogens, 2,2,4-Trimethylpentane (octane from standard gasoline)), and also potentially common organic solvents/reagents that are used in manufacturing (methacrylates, chloroform/methyl chloroform, acetylene, nitric acid), and how these different particulates effect AQI. Further, I want to see if there is a significant difference in air quality change across time in urban vs rural areas (not just if urban areas have better/worse air quality), based on specific particulates in the first part, and if there is any correlations between asthma rates in different areas and the levels of pollutants there (not across time because the asthma data does not have a time series attached to it).**

### Rough Plan for Data Analysis

1. (Already done as part of loading) Combine all EPA data files into one dataframe
1. Remove all rows that do not contain the parameters of interest (see above)
1. Remove columns that are mostly null/na/nan/missing values or are not interesting to the analysis
1. For locations that have multiple observations for a parameter in a year, average them and all statistics about the measurements for the year
1. Isolate a single metric for each parameter of interest (preferably "Observed Values"), preferably an overall average for the year and not the mean for daily maximums
1. Check for any correlations between the parameters of interest to other parameters of interest, and the AQI
1. Check changes in concentrations over time and see if there are any correlations between the changes in concentrations and the AQI
1. Geo-plot the change in AQI over time for each location, with an overlay showing the change in a normalization and aggregation of the parameters of interest
1. For 2020, geo-plot the AQI and the parameters of interest for each location and overlay the asthma rates for each location

In [1]:
# Imports
from itertools import chain
from typing import TYPE_CHECKING,  Literal

import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from matplotlib.axes import Axes
from matplotlib.figure import Figure

# this is a little bit of a hack to get typing hints for the projects function code since
# relative imports won't work properly and the runtime solution doesn't work
# for static analysis (type checking) since its not runtime
if TYPE_CHECKING:
    from .code import project_functions1 as utils
else:
    __import__("sys").path.append("./code")
    import project_functions1 as utils

In [2]:
# Setup seaborn/matplotlib theme information
sns.set_theme(style="darkgrid", font_scale=1.5) # type: ignore

In [10]:
__import__("importlib").reload(utils) and None

In [12]:
# Load the data using the function
df = utils.load_and_process(range(2011,2023))
# shapedata: gpd.GeoDataFrame = gpd.read_file(utils.PROJECT_ROOT / "data/raw/contiguous-usa.geojson")

In [14]:
df = (
    df.pivot_table(
        values=["Arithmetic Mean"],
        columns=["Parameter Name"],
        index=["CBSA Name", "Year"],
    )
    .droplevel(0, axis=1)
    .merge(
        df.drop(columns=["Arithmetic Mean", "Parameter Name"]),
        how="left",
        left_index=True,
        right_on=["CBSA Name", "Year"],
    )
).set_index(["Year", "CBSA Name"])

In [17]:
df = df.set_index(["Year", "CBSA Name"])

In [18]:
df = df.loc[2015].reset_index().set_index("CBSAFP")
df["Max AQI"] = df["Max AQI"].map(lambda x: x if 0 <= x <= 500 else 501 if x > 500 else 0)

In [25]:
df[
    [
        "Ammonium compounds",
        "Benzenes",
        "Carbon monoxide",
        "Heavy Metals",
        "Nitric oxides",
        "Nitrogen dioxide (NO2)",
        "Ozone",
        "Sulfur dioxide",
        "Various Particulates",
    ]
] = df[
    [
        "Ammonium compounds",
        "Benzenes",
        "Carbon monoxide",
        "Heavy Metals",
        "Nitric oxides",
        "Nitrogen dioxide (NO2)",
        "Ozone",
        "Sulfur dioxide",
        "Various Particulates",
    ]
].fillna(
    "Missing", axis="columns"
)


In [26]:
year = 2015
# fig = px.choropleth_mapbox(
#     df_tmp,
#     geojson=df_tmp.geometry,
#     locations=df_tmp.index,
#     color="Max AQI",
#     center={"lat": 37.0902, "lon": -95.7129},
#     mapbox_style="open-street-map",
#     zoom=3,
#     hover_name="NAME",
#     hover_data=[
#         "Days with AQI",
#         "90th Percentile AQI",
#         "Median AQI",
#         "Max AQI",
#         "Ammonium compounds",
#         "Benzenes",
#         "Carbon monoxide",
#         "Heavy Metals",
#         "Nitric oxides",
#         "Nitrogen dioxide (NO2)",
#         "Ozone",
#         "Sulfur dioxide",
#         "Various Particulates",
#         "Latitude",
#         "Longitude",
#     ],
#     title=f"Maximum Measured AQI values by CBSA for {year}. (Higher Numbers Are Worse)",
# )

# df_tmp.assign(text=lambda row : f"")

# fig = go.Figure(data=go.Choropleth(
#     locations=df_tmp.index,
#     z=df_tmp['Max AQI'],
#     # locationmode="geojson-id", #'USA-states',
#     geojson=df_tmp.geometry,
#     # colorscale='Reds',
#     # autocolorscale=False,
#     # text=df['text'], # hover text
#     marker_line_color='white', # line markers between states
#     colorbar_title="Max AQI (Higher is Worse)"
# ))
fig = px.choropleth(
    df,
    locations=df.index,
    color='Max AQI',
    geojson=df.geometry,
    # locationmode="geojson-id", #'USA-states',
    # colorscale='Reds',
    # autocolorscale=False,
    # text=df['text'], # hover text
    # marker_line_color='white', # line markers between states
    # colorbar_title="Max AQI (Higher is Worse)",
    hover_name="NAME",
    hover_data=[
        "Days with AQI",
        "90th Percentile AQI",
        "Median AQI",
        "Max AQI",
        "Ammonium compounds",
        "Benzenes",
        "Carbon monoxide",
        "Heavy Metals",
        "Nitric oxides",
        "Nitrogen dioxide (NO2)",
        "Ozone",
        "Sulfur dioxide",
        "Various Particulates",
        "Latitude",
        "Longitude",
    ],
)
fig.update_layout(
    # margin={"r": 0, "t": 50, "l": 0, "b": 0},
    title=f"Maximum Measured AQI values by CBSA for {year}. (Higher Numbers Are Worse)",
    geo=dict(
        scope="usa",
        projection=go.layout.geo.Projection(type="albers usa"),
        showlakes=True,  # lakes
        lakecolor="rgb(255, 255, 255)",
    ),
)
# hovertext = ""
# fig.update_traces()
fig.show()


NameError: name 'df_tmp' is not defined

In [ ]:
df_tmp.loc[39140]

In [ ]:
df.to_csv(utils.PROJECT_ROOT / "data/processed/processed1.csv")

In [ ]:
parameters = [
    "Max AQI",
    "Ozone",
    "Ammonium compounds",
    "Carbon monoxide",
    "Heavy Metals",
    "Nitric oxides",
    "Nitrogen dioxide (NO2)",
    "Sulfur dioxide",
    "Various Particulates",
    "Benzenes",
]


In [ ]:
test_geo["Max AQI"] = test_geo["Max AQI"].map(lambda value: value if 0 <= value <= 500 else 501 if value > 500 else 0)

In [ ]:
projection=gcrs.AlbersEqualArea(central_latitude=39.8282, central_longitude=-98.5795)

In [ ]:
years: list[Literal[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]] = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
figs: dict[Literal[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022], tuple[Figure, list[list[Axes]]]] = {}
for year in years:
    fig, axarr = plt.subplots(5, 2, figsize=(24, 28), subplot_kw={"projection": projection}, sharex=True, sharey=True, squeeze=True)
    for param, ax in zip(parameters,chain.from_iterable(axarr)):
        gplt.polyplot(shapedata, ax=ax)
        gplt.choropleth(
            df.dropna(subset=param).loc[2015],
            hue=param,
            legend=True,
            ax=ax,
            edgecolor="black",
            linewidth=1,
            cmap="autumn",
        )
        ax.set_title(label=param)
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
    fig.suptitle(f"Concentrations for various parameters divided by CBSA for {year} (Higher values are worse)", va="baseline")
    figs[year] = (fig, axarr)

In [ ]:
fig2, axarr2 = plt.subplots(6, 2, figsize=(24, 28), subplot_kw={"projection": projection}, sharex=True, sharey=True, squeeze=True)
year_mapping_info: dict[int, tuple[int, int]] = {
    2011: (0, 0),
    2012: (0, 1),
    2013: (1, 0),
    2014: (1, 1),
    2015: (2, 0),
    2016: (2, 1),
    2017: (3, 0),
    2018: (3, 1),
    2019: (4, 0),
    2020: (4, 1),
    2021: (5, 0),
    2022: (5, 1),
}
for year, ax in zip(list(range(2011,2023)),chain.from_iterable(axarr2)):
    ax = axarr2[year_mapping_info[year][0]][year_mapping_info[year][1]]
    gplt.polyplot(shapedata, projection=gcrs.AlbersEqualArea(central_latitude=39.8282, central_longitude=-98.5795), ax=ax)
    gplt.choropleth(
        test_geo.loc[year],
        hue="Max AQI",
        ax=ax,
        legend=True,
        edgecolor="black",
        linewidth=0.1,
        cmap="autumn",
        # legend_labels=legend_values
    )
    ax.set_title(f"{year}", fontsize=18)
fig2.tight_layout()
fig2.subplots_adjust(top=0.95)
fig2.suptitle(
    "Max AQI by CBSA in the continental US for 2011 to 2022",
    fontsize=18,
)
None

In [ ]:
# grouped = (
#     df.loc[df["Parameter Name"].isin(whitelist)]
#     .drop_duplicates(subset=["CBSA Name", "Year", "Parameter Name"])
#     .groupby(["CBSA Name", "Parameter Name"], as_index=False)
# )["Arithmetic Mean"].agg("mean").pivot(
#     values="Arithmetic Mean",
#     columns="Parameter Name",
#     index="CBSA Name",
# )
# corr = grouped.corr()

# # Generate a mask for the top right so we only generate one correlation per 
# # pair of factors.
# mask = np.triu(np.ones_like(corr, dtype=np.bool_))

# # Generate custom color map
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Create figure using seaborn:
# heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
# heatmap.set(title='Heatmap of Parameters')
# None